In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.utils as vutils
from torchsummary import summary
from tqdm import tqdm
import itertools
import time
import os
from os.path import isfile, join
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from google.colab import drive
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow
import random
import math
from collections import OrderedDict
from google.colab import drive
import sys
import subprocess

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
branch = 'master' 
!git clone https://github.com/OPHoperHPO/image-background-remove-tool.git -b $branch
%cd /content/image-background-remove-tool
!pip install -r requirements.txt
!cd ./tools && echo "all" | python setup.py
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'); print(device)

In [ ]:
preprocessing = "bbd-fastrcnn" #@param ["bbd-fastrcnn", "bbmd-maskrcnn", "None"] {allow-input: false}
model_name = "u2net" #@param ["u2net", "basnet", "u2netp", "mobile_net_model", "xception_model"] {allow-input: false}
postprocessing = "rtb-bnb" #@param ["rtb-bnb", "rtb-bnb2", "No"] {allow-input: false}


bashCommand = "python3 main.py -i /content/drive/MyDrive/BPTI_drone_archive/download.png -o /content/drive/MyDrive/BPTI_drone_archive/tikrinu.png -m $model_name -prep $preprocessing -postp $postprocessing"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

# !python3 main.py -i /content/drive/MyDrive/BPTI_drone_archive/download.png -o /content/drive/MyDrive/BPTI_drone_archive/tikrinu.png -m $model_name -prep $preprocessing -postp $postprocessing

In [ ]:
filenames =  os.listdir("/content/drive/MyDrive/BPTI_drone_archive/rembg/in/")
outputs = []
outputs_white = []
bash_commands = []
for i in range(len(filenames)):
  outputs.append("/content/drive/MyDrive/BPTI_drone_archive/rembg/out/" + filenames[i][0:-3] + "png")
  outputs_white.append("/content/drive/MyDrive/BPTI_drone_archive/rembg/out_white/" + filenames[i])
  filenames[i] = "/content/drive/MyDrive/BPTI_drone_archive/rembg/in/" + filenames[i]
  outputs_white.append("/content/drive/MyDrive/BPTI_drone_archive/rembg/out_white/" + filenames[i])
  bash_commands.append("python3 main.py -i " + filenames[i] + " -o " + outputs[i] + " -m $model_name -prep $preprocessing -postp $postprocessing")

FileNotFoundError: ignored

In [ ]:
preprocessing = "bbd-fastrcnn" #@param ["bbd-fastrcnn", "bbmd-maskrcnn", "None"] {allow-input: false}
model_name = "u2net" #@param ["u2net", "basnet", "u2netp", "mobile_net_model", "xception_model"] {allow-input: false}
postprocessing = "rtb-bnb" #@param ["rtb-bnb", "rtb-bnb2", "No"] {allow-input: false}

for i in range(len(filenames)):
  bashCommand = bash_commands[i]
  process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
  output, error = process.communicate()

In [ ]:
for i in range(len(filenames)):
  img = cv2.imread(outputs[i], cv2.IMREAD_UNCHANGED)
  trans_mask = img[:,:,3] == 0
  img[trans_mask] = [255, 255, 255, 255]
  img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
  cv2.imwrite(outputs_white[i], img)

In [ ]:
A_paths = []

for files in os.listdir(dir_A):
  A_paths.append(os.path.join(dir_A, files)) 

A_paths = sorted(A_paths)

sorted_test = A_paths

In [ ]:
a = 255

BLUR = 21
CANNY_THRESH_1 = a
CANNY_THRESH_2 = a / 3
MASK_DILATE_ITER = 10
MASK_ERODE_ITER = 10
MASK_COLOR = (0.0,0.0,1.0) # In BGR format


filename = A_paths[5]
img = cv2.imread(filename)
xmin, ymin, xmax, ymax = our_lib.square_bboxes(df[str(filename.split('/')[7])]['bboxes'][0])  
img = img[ymin:ymax, xmin:xmax]
cv2_imshow(img)

edges = cv2.Canny(img,CANNY_THRESH_1,CANNY_THRESH_2)
cv2_imshow(edges)
edges = cv2.dilate(edges, None)
cv2_imshow(edges)
edges = cv2.erode(edges, None)
cv2_imshow(edges)

#-- Find contours in edges, sort by area ---------------------------------------------
contour_info = []
contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

for c in contours:
    contour_info.append((
        c,
        cv2.isContourConvex(c),
        cv2.contourArea(c),
    ))
contour_info = sorted(contour_info, key=lambda c: c[2], reverse=True)
max_contour = contour_info[0]


mask = np.zeros(edges.shape)
cv2.fillConvexPoly(mask, max_contour[0], (255))


mask = cv2.dilate(mask, None, iterations=MASK_DILATE_ITER)
mask = cv2.erode(mask, None, iterations=MASK_ERODE_ITER)
mask = cv2.GaussianBlur(mask, (BLUR, BLUR), 0)
mask_stack = np.dstack([mask]*3)  

mask_stack  = mask_stack.astype('float32') / 255.0        
img         = img.astype('float32') / 255.0  

masked = (mask_stack * img) + ((1-mask_stack) * MASK_COLOR) 
masked = (masked * 255).astype('uint8')                     

cv2_imshow(masked)

c_red, c_green, c_blue = cv2.split(img)

# img_a = cv2.merge((c_red, c_green, c_blue, mask.astype('float32') / 255.0))
img_a = cv2.merge((c_blue, c_green, c_red,  mask.astype('float32') / 255.0))


plt.imshow(img_a)

# print(img_a.shape)
# print(img_a[:][:][3][0][0][0].shape)
# plt.imshow(np.transpose(img_a[:][:][2], (2, 1, 0)))